In [1]:
# import realted module
import pandas as pd
import numpy as np

route46A = pd.read_csv('F:/result_of_46A.csv')

In [2]:
route46A.head()

,Unnamed: 0,arrive_time,clouds_all,day,dayofweek,dt,duration,end_point,humidity,month,pressure,start_point,temp,weather_description,wind_deg,wind_speed
0,0,64279,20.0,24,1,1464112279,148.0,808.0,67.0,5,1022.0,807,287.67,few clouds,80.0,6.0
1,1,64427,20.0,24,1,1464112427,112.0,809.0,67.0,5,1022.0,808,287.67,few clouds,80.0,6.0
2,2,64539,20.0,24,1,1464112539,126.0,810.0,67.0,5,1022.0,809,287.67,few clouds,80.0,6.0
3,3,64665,20.0,24,1,1464112665,75.0,811.0,67.0,5,1022.0,810,287.67,few clouds,80.0,6.0
4,4,64740,20.0,24,1,1464112740,41.0,812.0,67.0,5,1022.0,811,287.67,few clouds,80.0,6.0


In [3]:
route46A.shape

(871584, 16)

In [22]:
continouos = ['temp','arrive_time']
df_continouos = route46A[continouos]
df_continouos.head()

from scipy.stats import zscore
df_continouos = df_continouos.apply(zscore)

In [23]:
# Binary Coding of weather
binary_weather = pd.get_dummies(route46A['weather_description'])
# Binary Coding of endstop
binary_end_stop = pd.get_dummies(route46A['end_point'])
# Binary Coding of startstop
binary_start_stop = pd.get_dummies(route46A['start_point'])
# Binary Coding of dayofweek
binary_dayofweek = pd.get_dummies(route46A['dayofweek'])


df = pd.concat([df_continouos, binary_weather, binary_end_stop, binary_start_stop, binary_dayofweek], axis=1)
df.head()

,temp,arrive_time,Sky is Clear,broken clouds,few clouds,fog,heavy intensity rain,light intensity drizzle,light intensity drizzle rain,light intensity shower rain,...,4962,6059,7353,0,1,2,3,4,5,6
0,1.175395,0.618607,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,1.175395,0.627293,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,1.175395,0.633867,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,1.175395,0.641262,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,1.175395,0.645663,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [24]:
df_new = pd.concat([df, route46A['duration']], axis=1)
df_new = df_new.sample(frac=1)
df_train = df_new.head(610108)
df_test = df_new.tail(261476)
df_train.head()

,temp,arrive_time,Sky is Clear,broken clouds,few clouds,fog,heavy intensity rain,light intensity drizzle,light intensity drizzle rain,light intensity shower rain,...,6059,7353,0,1,2,3,4,5,6,duration
335005,0.785022,-0.905159,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,27.0
728068,0.385675,-0.662300,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,251.0
762630,0.809701,-0.907389,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,240.0
803166,-0.549874,1.259677,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,43.0
53668,-0.076491,1.286029,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,91.0


In [25]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
X = df_train.drop('duration', axis = 1)
y = df_train['duration']

model = model.fit(X, y)
model.get_params()

{'copy_X': True, 'fit_intercept': True, 'n_jobs': 1, 'normalize': False}

In [26]:
model.score(X, y)

0.46875791195657784

In [27]:
test_x = df_test.drop('duration', axis = 1)
test_y = df_test['duration']
model.score(test_x, test_y)

0.4498366568330818

In [28]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(n_estimators=50, min_samples_split=5, n_jobs=-1)
regr.fit(X, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=5,
           min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [29]:
regr.score(X, y)

0.8406810035159866

In [30]:
regr.score(test_x, test_y)

0.3925114953663984

In [32]:
from sklearn.svm import SVR

svr_model = SVR(C=1000,gamma=0.3,kernel='rbf')
svr_model.fit(X.head(20000), y.head(20000))

SVR(C=1000, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.3,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
svr_model.score(X, y)